<a id="readme-top"></a>
# LP read mps notebook

## 📚 Table of Content
- [Reading File to String](#reading-file-to-string)
- [Reading MPS File From String](#reading-mps-file-from-string)
    - [Original Function](#original-function)
    - [Adding Multiple Column Support](#adding-multiple-column-support)
- [Unit Testing](#unit-testing)
    - [Manual Approach](#manual-approach)
- [Testing with JuMP](#testing-with-jump)
    - [Using JuMP MathOptInterface as a reference for testing](#using-jump-mathoptinterface-as-a-reference-for-testing)


## Discription Of MPS Format

## MPS File Format Documentation

The **MPS (Mathematical Programming System)** file format is a standardized format used for representing linear programming (LP) and mixed-integer programming (MIP) problems. Developed by IBM in the 1970s, it is widely used for exchanging problem instances between different optimization software.

### Structure of an MPS File

An MPS file is a plain text file that is divided into several sections. Each section begins with a keyword and is followed by relevant data:

#### 1. **NAME Section**
   - **Purpose**: Identifies the name of the problem.
   - **Format**: `NAME <problem_name>`

#### 2. **ROWS Section**
   - **Purpose**: Defines the objective function and constraints.
   - **Format**: Each row is identified by a type and name.
   - **Row Types**:
     - `N`: The objective function.
     - `L`: Less than or equal to (≤) constraints.
     - `G`: Greater than or equal to (≥) constraints.
     - `E`: Equality (=) constraints.

#### 3. **COLUMNS Section**
   - **Purpose**: Specifies the coefficients of variables in the objective function and constraints.
   - **Format**: Each line in this section specifies a variable and its coefficient in one or two constraints or the objective function.
   - **Example**:
     ```
     COLUMN_NAME ROW_NAME coefficient
     ```

#### 4. **RHS Section**
   - **Purpose**: Specifies the right-hand side (RHS) values for the constraints.
   - **Format**: Similar to the COLUMNS section but with RHS values.

#### 5. **BOUNDS Section**
   - **Purpose**: Specifies the bounds on the variables.
   - **Types of Bounds**:
     - `LO`: Lower bound.
     - `UP`: Upper bound.
     - `FX`: Fixed variable (both lower and upper bound are equal).
     - `FR`: Free variable (no bounds).
   - **Integer Variable Definition Using Markers**
   In addition to the standard bound types, some solvers use marker tags to define integer variables. The `MARKER 'INTORG' 'MARKER'` and `MARKER 'INTEND' 'MARKER'` approach is used to indicate blocks of variables that should be treated as integers.



#### Example Usage:

```mps
MARKER 'INTORG' 'MARKER'
    X1    ROW1    1
    X2    ROW2    2
    X3    ROW3    3
MARKER 'INTEND' 'MARKER'
   - **Format**: 
     ```
     BOUND_TYPE BOUND_NAME VARIABLE_NAME value
     ```

### 6. **ENDATA Section**
   - **Purpose**: Marks the end of the MPS file.
   - **Format**: The keyword `ENDATA`.

## Example MPS File

## References

- [MPS Format - Gurobi Documentation](https://www.gurobi.com/documentation/9.1/refman/mps_format.html)
- [MPS File Format - IBM Documentation](https://www.ibm.com/docs/en/icos/12.9.0?topic=formats-mps-file-format)




In [1]:
using LinearAlgebra
using SparseArrays
using Random
using ArgParse
using DataStructures
using Test

# JuMP
using JuMP
using MathOptInterface
const MOI = MathOptInterface

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_constants
using lp_utils
using lp_problem


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Reading File to String

The following function reads a file and outputs a string. It is an important step because we can use string methods and technequies to access data.

In [2]:
function read_file_to_string(file_path::String)
    file_string = open(file_path, "r") do f
        read(f, String)
    end
    return file_string
end

read_file_to_string (generic function with 1 method)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Reading MPS File From String

In [ ]:
function read_mps_from_file(file_path::String)
    mps_string = open(file_path, "r") do f
        read(f, String)
    end
    return read_mps_from_string(mps_string)
end

<p align="right">(<a href="#readme-top">back to top</a>)</p>

### Original Function

In [ ]:
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end

### Adding Multiple Column Support

In [ ]:
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end

In [ ]:

# Test
mps_filename = "../../benchmarks/mps_files/test.mps"
test_string = read_file_to_string(mps_filename)

lp = read_mps_from_string_multicolumns(test_string)
println(lp)

println("")

dense_lp = Matrix(lp.A)
println("c: ", lp.c)
println("b: ", lp.b)
println("vars: ", lp.vars)
println("constraint_types: ", lp.constraint_types)
println("A: ", dense_lp)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Adding MIP Support

In [3]:
# struct MIPProblem
#     is_minimize::Bool  # True if the objective is to minimize
#     c::Vector{Float64}  # Objective function coefficients
#     A::SparseMatrixCSC{Float64, Int64}  # Constraint matrix
#     b::Vector{Float64}  # Right-hand side of constraints
#     l::Vector{Float64}  # Variable lower bounds
#     u::Vector{Float64}  # Variable upper bounds
#     vars::Vector{String}  # Variable names
#     constraint_types::Vector{Char}  # Constraint types
#     variable_types::Dict{String, Symbol}  # Field for variable types
# end

struct MIPProblem
    is_minimize::Bool  # True if the objective is to minimize
    c::Vector{Float64}  # Objective function coefficients
    A::SparseMatrixCSC{Float64, Int64}  # Constraint matrix
    b::Vector{Float64}  # Right-hand side of constraints
    l::Vector{Float64}  # Variable lower bounds
    u::Vector{Float64}  # Variable upper bounds
    vars::Vector{String}  # Variable names
    variable_types::Vector{Symbol}  # Array of variable types, same length as vars
    constraint_types::Vector{Char}  # Constraint types
end


In [4]:
MPS_MIP_Example = """
NAME          TESTPROB
ROWS
 N  OBJ
 L  ROW1
 G  ROW2
COLUMNS
    X1    OBJ     1
    X1    ROW1    2
    X2    OBJ     1
    X2    ROW2    1
    MARKER 'INTORG' 'MARKER'
    X3    OBJ     1
    X3    ROW1    1
    MARKER 'INTEND' 'MARKER'
RHS
    RHS1   ROW1    5
    RHS1   ROW2    10
BOUNDS
 UP BND1  X1      10
 LO BND1  X1      0
 UP BND1  X2      1
 LO BND1  X2      0
 UP BND1  X3      15
 LO BND1  X3      0
ENDATA
"""


"NAME          TESTPROB\nROWS\n N  OBJ\n L  ROW1\n G  ROW2\nCOLUMNS\n    X1    OBJ     1\n    X1    ROW1    2\n    X2    OBJ     1\n    X2    ROW2    1\n    MARKER 'INTORG' 'MARKER'\n    X3    OBJ     1\n    X3    ROW1    1\n    MARKER 'INTEND' 'MARKER'\nRHS\n    RHS1   ROW1    5\n    RHS1   ROW2    10\nBOUNDS\n UP BND1  X1      10\n LO BND1  X1      0\n UP BND1  X2      1\n LO BND1  X2      0\n UP BND1  X3      15\n LO BND1  X3      0\nENDATA\n"

In [6]:
function read_mps_from_string_mip(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true
    objective_set = false
    in_integer_block = false

    # New array to keep track of variable types (continuous by default)
    variable_types = OrderedDict{String, Symbol}()

    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            if words[1] == "MARKER"
                if words[2] == "'INTORG'"
                    in_integer_block = true
                elseif words[2] == "'INTEND'"
                    in_integer_block = false
                end
                continue
            end

            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
                # Set variable type based on whether we are in an integer block
                variable_types[col_name] = in_integer_block ? :integer : :continuous
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, FX, BV, LI, UI, SC, SI
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR, MI, PL
                bound_type, _, var_name = words
                value = Inf
            end

            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end

            if bound_type in ["BV", "LI", "UI", "SC", "SI"]
                # Determine variable type based on the bound type
                if bound_type == "BV"
                    variable_types[var_name] = :binary
                elseif bound_type == "LI" || bound_type == "UI"
                    variable_types[var_name] = :integer
                elseif bound_type == "SC"
                    variable_types[var_name] = :semi_continuous
                elseif bound_type == "SI"
                    variable_types[var_name] = :semi_integer
                end
            end

            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to MIPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    # Convert variable_types from OrderedDict to an array
    variable_types_array = [variable_types[var] for var in vars]

    # Construct the MIPProblem struct
    mip_problem = MIPProblem(
        is_minimize,
        c,
        A,
        b,
        lb,
        ub,
        vars,
        variable_types_array,
        constraint_types,
    )

    return mip_problem
end


read_mps_from_string_mip (generic function with 1 method)

In [9]:
mps_string = MPS_MIP_Example

#mps_string = read_file_to_string("../../benchmarks/mps_files/simple.mps")
lp_mip = read_mps_from_string_mip(mps_string)
#lp_mip = read_mps_from_string_multicolumns(mps_string)

println("Objective function coefficients: ", lp_mip.c)
println("Constraint matrix: ", lp_mip.A)
println("Right-hand side: ", lp_mip.b)
println("Lower bounds: ", lp_mip.l)
println("Upper bounds: ", lp_mip.u)
println("Variable types: ", lp_mip.variable_types)
println("constraint types types: ", lp_mip.constraint_types)


Objective function coefficients: [1.0, 1.0, 1.0]
Constraint matrix: sparse([1, 2, 1], [1, 2, 3], [2.0, -1.0, 1.0], 2, 3)
Right-hand side: [5.0, -10.0]
Lower bounds: [0.0, 0.0, 0.0]
Upper bounds: [10.0, 1.0, 15.0]
Variable types: [:continuous, :continuous, :integer]
constraint types types: ['L', 'G']


In [34]:
mps_filepath = "../../benchmarks/mps_files/simple.mps"
mps_string = read_file_to_string(mps_filepath)

println(read_mps_from_string_mip(mps_string))
println(read_mps_with_JuMP_MIP(mps_filepath))
#lp_mip

MIPProblem(false, [3.0, 2.0], sparse([1, 2, 1, 3], [1, 1, 2, 2], [1.0, 1.0, 1.0, 1.0], 3, 2), [4.0, 2.0, 3.0], [0.0, 0.0], [Inf, Inf], ["x1", "x2"], [:continuous, :continuous], ['L', 'L', 'L'])
MIPProblem(false, [3.0, 2.0], sparse([1, 2, 1, 3], [1, 1, 2, 2], [1.0, 1.0, 1.0, 1.0], 3, 2), [4.0, 2.0, 3.0], [0.0, 0.0], [Inf, Inf], ["x1", "x2"], [:continuous, :continuous], ['L', 'L', 'L'])


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Unit Testing

### Manual Approach

In [ ]:
mps_filename = "../../benchmarks/mps_files/test.mps"
lp_string = read_file_to_string(mps_filename)
println(lp_string)

In [ ]:
# Define the expected outputs for comparison
expected_c = [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
expected_b = [200.0, 250.0, 0.0, 0.0, 0.0]
expected_vars = ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
expected_constraint_types = ['L', 'L', 'L', 'G', 'E']

# Define the expected A matrix (as a sparse matrix for comparison)
expected_A = [
    [1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -6.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -3.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, -1.0]
]

# The map(x -> x == -0.0 ? 0.0 : x, lp.) is used to ensure no non zeros.

# Write the test cases
@testset "MPS Parsing Tests" begin
    # Test the objective function coefficients
    @test map(x -> x == -0.0 ? 0.0 : x, lp.c) == expected_c
    
    # Test the right-hand side vector
    @test map(x -> x == -0.0 ? 0.0 : x, lp.b) == expected_b
    
    # Test the variable names
    @test map(x -> x == -0.0 ? 0.0 : x, lp.vars) == expected_vars
    
    # Test the constraint types
    @test map(x -> x == -0.0 ? 0.0 : x, lp.constraint_types) == expected_constraint_types
    
    # Test the constraint matrix (A)
    #@test Matrix(lp.A) == expected_A # This indicates an issue.
end

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Testing with JuMP

Here I look at the multicolumned example of test.mps to check if the output is as expected.

In [ ]:
mps_filename = "../../benchmarks/mps_files/test.mps"
test_string = read_file_to_string(mps_filename)
println(test_string)

In [ ]:
# Define the expected outputs for comparison
expected_c = [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
expected_b = [200.0, 250.0, 0.0, 0.0, 0.0]
expected_vars = ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
expected_constraint_types = ['L', 'L', 'L', 'G', 'E']

# Define the expected A matrix (as a sparse matrix for comparison)
expected_A = [
    [1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -6.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -3.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, -1.0]
]

# The map(x -> x == -0.0 ? 0.0 : x, lp.) is used to ensure no non zeros.

# Write the test cases
@testset "MPS Parsing Tests" begin
    # Test the objective function coefficients
    @test map(x -> x == -0.0 ? 0.0 : x, lp.c) == expected_c
    
    # Test the right-hand side vector
    @test map(x -> x == -0.0 ? 0.0 : x, lp.b) == expected_b
    
    # Test the variable names
    @test map(x -> x == -0.0 ? 0.0 : x, lp.vars) == expected_vars
    
    # Test the constraint types
    @test map(x -> x == -0.0 ? 0.0 : x, lp.constraint_types) == expected_constraint_types
    
    # Test the constraint matrix (A)
    #@test Matrix(lp.A) == expected_A
end

### Using JuMP MathOptInterface as a reference for testing

In [13]:
"""
    read_and_process_mps(file_path::String) -> LPProblem

Reads an MPS file from the provided file path, processes it into a MathOptInterface model, 
and returns an LPProblem struct with all the relevant details.

# Arguments
- `file_path::String`: The file path to the MPS file.

# Returns
- `LPProblem`: A struct representing the linear programming problem.
"""
function read_mps_with_JuMP(file_path::String)
    # Create a utility model
    model = MOI.Utilities.Model{Float64}()

    # Read the MPS file into the model
    MOI.read_from_file(model, file_path)

    # Extract variables
    variables = MOI.get(model, MOI.ListOfVariableIndices())

    # Extract variable names
    variable_names = [MOI.get(model, MOI.VariableName(), var) for var in variables]

    # Extract the objective function (assumes a linear objective)
    objective_function = MOI.get(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
    objective_coeffs = zeros(Float64, length(variables))

    # Populate the objective coefficients array
    for term in objective_function.terms
        objective_coeffs[term.variable.value] = term.coefficient
    end

    # Determine whether it's a minimization or maximization problem
    is_minimize = MOI.get(model, MOI.ObjectiveSense()) == MOI.MIN_SENSE

    # Separate constraints by type
    less_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
    greater_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
    equal_to_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

    constraint_matrix_rows = Int64[]
    constraint_matrix_cols = Int64[]
    constraint_matrix_vals = Float64[]
    rhs_values = Float64[]
    constraint_types = Char[]

    # Process LessThan constraints
    for con in less_than_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.upper)
        push!(constraint_types, 'L')
    end

    # Process GreaterThan constraints
    for con in greater_than_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.lower)
        push!(constraint_types, 'G')
    end

    # Process EqualTo constraints
    for con in equal_to_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.value)
        push!(constraint_types, 'E')
    end

    # Convert to sparse matrix
    constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(rhs_values), length(variables))

    # Define lower and upper bounds
    lower_bounds = fill(0.0, length(variables))  # Default lower bounds (0.0)
    upper_bounds = fill(Inf, length(variables))  # Default upper bounds (Inf)

    # Construct the LPProblem struct
    lp = LPProblem(
        is_minimize,
        objective_coeffs,
        constraint_matrix,
        rhs_values,
        lower_bounds,
        upper_bounds,
        variable_names,
        constraint_types
    )

    return lp
end


read_mps_with_JuMP

In [ ]:
using JuMP

# Create a new JuMP model using the default solver
model = Model()

# Add variables of different types
@variable(model, x)         # Continuous by default
@variable(model, y, Int)    # Integer
@variable(model, z, Bin)    # Binary

# Get the list of variables
variables = [x, y, z]

# Function to determine the type of variable
function get_variable_type(var)
    if is_binary(var)
        return "Binary"
    elseif is_integer(var)
        return "Integer"
    else
        return "Continuous"
    end
end

# Retrieve the types of the variables
variable_types = [get_variable_type(var) for var in variables]

# Output the types
println("Variable Types: ", variable_types)


In [31]:
# Function to determine the type of variable
function get_variable_type(var)
    if is_binary(var)
        return "Binary"
    elseif is_integer(var)
        return "integer"
    else
        return "continuous"
    end
end


get_variable_type (generic function with 1 method)

In [32]:
function read_mps_with_JuMP_MIP(file_path::String)
    # Create a JuMP model
    model = Model()

    # Read the MPS file into the model
    MOI.read_from_file(model.moi_backend, file_path)

    # Extract variables
    variables = all_variables(model)

    # Extract variable names
    variable_names = [name(var) for var in variables]

    # Extract the objective function (assumes a linear objective)
    objective_function = MOI.get(model.moi_backend, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
    objective_coeffs = zeros(Float64, length(variables))

    # Populate the objective coefficients array
    for term in objective_function.terms
        objective_coeffs[term.variable.value] = term.coefficient
    end

    # Determine whether it's a minimization or maximization problem
    is_minimize = MOI.get(model.moi_backend, MOI.ObjectiveSense()) == MOI.MIN_SENSE

    # Separate constraints by type
    less_than_constraints = MOI.get(model.moi_backend, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
    greater_than_constraints = MOI.get(model.moi_backend, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
    equal_to_constraints = MOI.get(model.moi_backend, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

    constraint_matrix_rows = Int64[]
    constraint_matrix_cols = Int64[]
    constraint_matrix_vals = Float64[]
    rhs_values = Float64[]
    constraint_types = Char[]

    # Process LessThan constraints
    for con in less_than_constraints
        func = MOI.get(model.moi_backend, MOI.ConstraintFunction(), con)
        set = MOI.get(model.moi_backend, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.upper)
        push!(constraint_types, 'L')
    end

    # Process GreaterThan constraints
    for con in greater_than_constraints
        func = MOI.get(model.moi_backend, MOI.ConstraintFunction(), con)
        set = MOI.get(model.moi_backend, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.lower)
        push!(constraint_types, 'G')
    end

    # Process EqualTo constraints
    for con in equal_to_constraints
        func = MOI.get(model.moi_backend, MOI.ConstraintFunction(), con)
        set = MOI.get(model.moi_backend, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.value)
        push!(constraint_types, 'E')
    end

    # Convert to sparse matrix
    constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(rhs_values), length(variables))

    # Define lower and upper bounds
    lower_bounds = fill(0.0, length(variables))  # Default lower bounds (0.0)
    upper_bounds = fill(Inf, length(variables))  # Default upper bounds (Inf)

    # Variable types array
    variable_types = Vector{Symbol}(undef, length(variables))

    # Determine the type of each variable using the provided get_variable_type function
    for (i, var) in enumerate(variables)
        variable_types[i] = Symbol(get_variable_type(var))
    end

    # Construct the MIPProblem struct
    mip_problem = MIPProblem(
        is_minimize,
        objective_coeffs,
        constraint_matrix,
        rhs_values,
        lower_bounds,
        upper_bounds,
        variable_names,
        variable_types,
        constraint_types,
    )

    return mip_problem
end


read_mps_with_JuMP_MIP (generic function with 1 method)

In [14]:
read_mps_with_JuMP("../../benchmarks/mps_files/test.mps")

LPProblem(false, [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0], sparse([1, 3, 1, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 3], [1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6], [10.8, 8.8, 8.1, 6.1, 3.0, 1.0, 2.0, 5.2, 1.0, 4.2, 6.0, 1.0, 5.0, -4.0, -6.0], 5, 6), [200.0, 250.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf], ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"], ['L', 'L', 'L', 'G', 'E'])

In [15]:
read_mps_with_JuMP_MIP("../../benchmarks/mps_files/test.mps")

MIPProblem(false, [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0], sparse([1, 3, 1, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 3], [1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6], [10.8, 8.8, 8.1, 6.1, 3.0, 1.0, 2.0, 5.2, 1.0, 4.2, 6.0, 1.0, 5.0, -4.0, -6.0], 5, 6), [200.0, 250.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf], ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous, :Continuous], ['L', 'L', 'L', 'G', 'E'])

In [ ]:
function test_read_mps_from_string_multicolumns(mps_file::String)
    @testset "Test read_mps_from_string_multicolumns function with $mps_file" begin
        
        # Use the read_file_to_string function to read the MPS file content as a string
        test_string = read_file_to_string(mps_file)

        # Use the function you wrote to parse the MPS string
        # lp_output = read_mps_from_string_multicolumns(test_string)
        lp_output = read_mps_from_string_mip(test_string)

        # Use MOI to create the expected model
        model = MOI.Utilities.Model{Float64}()
        MOI.read_from_file(model, mps_file)

        # Extract variables and their names using MOI
        variables = MOI.get(model, MOI.ListOfVariableIndices())
        variable_names = [MOI.get(model, MOI.VariableName(), var) for var in variables]

        # Extract the objective coefficients using MOI
        objective_function = MOI.get(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
        expected_objective_coeffs = zeros(Float64, length(variables))
        for term in objective_function.terms
            expected_objective_coeffs[term.variable.value] = term.coefficient
        end

        # Extract the RHS and constraint matrix using MOI
        less_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
        greater_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
        equal_to_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

        constraint_matrix_rows = Int[]
        constraint_matrix_cols = Int[]
        constraint_matrix_vals = Float64[]
        expected_rhs = Float64[]
        expected_constraint_types = Char[]

        for con in less_than_constraints
            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)
            for term in func.terms
                push!(constraint_matrix_rows, con.value)
                push!(constraint_matrix_cols, term.variable.value)
                push!(constraint_matrix_vals, term.coefficient)
            end
            push!(expected_rhs, set.upper)
            push!(expected_constraint_types, 'L')
        end

        for con in greater_than_constraints
            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)
            for term in func.terms
                push!(constraint_matrix_rows, con.value)
                push!(constraint_matrix_cols, term.variable.value)
                push!(constraint_matrix_vals, term.coefficient)
            end
            push!(expected_rhs, set.lower)
            push!(expected_constraint_types, 'G')
        end

        for con in equal_to_constraints
            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)
            for term in func.terms
                push!(constraint_matrix_rows, con.value)
                push!(constraint_matrix_cols, term.variable.value)
                push!(constraint_matrix_vals, term.coefficient)
            end
            push!(expected_rhs, set.value)
            push!(expected_constraint_types, 'E')
        end

        expected_constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(expected_rhs), length(variables))

        # Define expected results
        expected_vars = variable_names
        expected_c = expected_objective_coeffs

        # Convert constraint matrix to dense for easier testing
        dense_expected_A = Matrix(expected_constraint_matrix)

        # Assertions to compare function output with MOI expected output
        println("Testing variable names...")
        @test lp_output.vars == expected_vars

        println("Testing objective coefficients...")
        @test lp_output.c == expected_c

        println("Testing RHS values...")
        @test lp_output.b == expected_rhs

        println("Testing constraint types...")
        @test lp_output.constraint_types == expected_constraint_types

        println("Testing constraint matrix...")
        @test Matrix(lp_output.A) == dense_expected_A

    end
end


In [29]:
mps_filepath = "../../benchmarks/mps_files/simple.mps"
test_mps_parsing_consistency(mps_filepath)


# Run the test with a specific MPS file
test_filepath = mps_filename
test_read_mps_from_string_multicolumns(test_filepath)

# 
# mps_mip_filename = "../../benchmarks/mps_files/MIP-test.mps"
# test_read_mps_from_string_multicolumns(mps_mip_filename)

Testing is_minimize...
Testing objective function coefficients...
Testing constraint matrix...
Testing RHS values...
Testing lower bounds...
Testing upper bounds...
Testing variable names...
Testing constraint types...
Testing variable types...
Test consistency between MPS parsing functions with ../../benchmarks/mps_files/simple.mps: Test Failed at In[26]:37
  Expression: lp_mip_1.variable_types == lp_mip_2.variable_types
   Evaluated: [:continuous, :continuous] == [:Continuous, :Continuous]
Stacktrace:
 [1] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:445 [inlined]
 [2] macro expansion
   @ ./In[26]:37 [inlined]
 [3] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:1283 [inlined]
 [4] test_mps_parsing_consistency(mps_file::String)
   @ Main ./In[26]:5
Test Summary:                                                                             | Pass  Fai

LoadError: [91mSome tests did not pass: 8 passed, 1 failed, 0 errored, 0 broken.[39m

In [26]:
function test_mps_parsing_consistency(mps_file::String)
    @testset "Test consistency between MPS parsing functions with $mps_file" begin
        
        # Read the MPS file content as a string
        mps_string = read_file_to_string(mps_file)

        # Parse the MPS string using both functions
        lp_mip_1 = read_mps_from_string_mip(mps_string)
        lp_mip_2 = read_mps_with_JuMP_MIP(mps_file)

        # Compare the outputs of both functions
        println("Testing is_minimize...")
        @test lp_mip_1.is_minimize == lp_mip_2.is_minimize

        println("Testing objective function coefficients...")
        @test lp_mip_1.c == lp_mip_2.c

        println("Testing constraint matrix...")
        @test lp_mip_1.A == lp_mip_2.A

        println("Testing RHS values...")
        @test lp_mip_1.b == lp_mip_2.b

        println("Testing lower bounds...")
        @test lp_mip_1.l == lp_mip_2.l

        println("Testing upper bounds...")
        @test lp_mip_1.u == lp_mip_2.u

        println("Testing variable names...")
        @test lp_mip_1.vars == lp_mip_2.vars

        println("Testing constraint types...")
        @test lp_mip_1.constraint_types == lp_mip_2.constraint_types

        println("Testing variable types...")
        @test lp_mip_1.variable_types == lp_mip_2.variable_types

    end
end



test_mps_parsing_consistency (generic function with 1 method)

In [28]:
# Define the MPS file path
mps_filepath = "../../benchmarks/mps_files/simple.mps"

# Call the test function
test_mps_parsing_consistency(mps_filepath)

Testing is_minimize...
Testing objective function coefficients...
Testing constraint matrix...
Testing RHS values...
Testing lower bounds...
Testing upper bounds...
Testing variable names...
Testing constraint types...
Testing variable types...
Test consistency between MPS parsing functions with ../../benchmarks/mps_files/simple.mps: Test Failed at In[26]:37
  Expression: lp_mip_1.variable_types == lp_mip_2.variable_types
   Evaluated: [:continuous, :continuous] == [:Continuous, :Continuous]
Stacktrace:
 [1] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:445 [inlined]
 [2] macro expansion
   @ ./In[26]:37 [inlined]
 [3] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:1283 [inlined]
 [4] test_mps_parsing_consistency(mps_file::String)
   @ Main ./In[26]:5
Test Summary:                                                                             | Pass  Fai

LoadError: [91mSome tests did not pass: 8 passed, 1 failed, 0 errored, 0 broken.[39m

In [33]:
mps_filename = "../../benchmarks/mps_files/test.mps"
test_string = read_file_to_string(mps_filename)

#test_filepath = mps_filename
test_read_mps_from_string_mip(mps_filename)

Test read_mps_from_string_mip function with ../../benchmarks/mps_files/test.mps: Error During Test at In[19]:2
  Got exception outside of a @test
  UndefVarError: VariableType not defined
  Stacktrace:
    [1] macro expansion
      @ ./In[19]:84 [inlined]
    [2] macro expansion
      @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:1283 [inlined]
    [3] test_read_mps_from_string_mip(mps_file::String)
      @ Main ./In[19]:5
    [4] top-level scope
      @ In[33]:5
    [5] eval
      @ ./boot.jl:373 [inlined]
    [6] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
      @ Base ./loading.jl:1196
    [7] softscope_include_string(m::Module, code::String, filename::String)
      @ SoftGlobalScope ~/.julia/packages/SoftGlobalScope/u4UzH/src/SoftGlobalScope.jl:65
    [8] execute_request(socket::ZMQ.Socket, msg::IJulia.Msg)
      @ IJulia ~/.julia/packages/IJulia/AQu2H/src/execute_request.jl:67
    [9

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.[39m